*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Parte 3: Compromisos FRI (FRI Commitments)


### Cargue la sesión anterior
Ejecute la siguiente celda para cargar las variables relevantes. Como es habitual, tardará un poco en ejecutarse.

In [ ]:
from channel import Channel
from field import FieldElement
from merkle import MerkleTree
from polynomial import interpolate_poly, Polynomial
from tutorial_sessions import part1, part2

cp, cp_eval, cp_merkle, channel, eval_domain = part2()
print("Éxito")

## Plegado FRI (FRI Folding)

Nuestro objetivo en esta parte es construir las capas FRI y comprometernos con ellas. 
<br>Para obtener cada capa necesitamos:
1. Generar un dominio para la capa (a partir del dominio de la capa anterior).
2. Generar un polinomio para la capa (a partir del polinomio y dominio de la capa anterior)
3. Evaluar dicho polinomio en dicho dominio - **esta es la siguiente capa FRI**.

### Generación de dominios

El primer dominio FRI es simplemente el `eval_domain` que ya se generó en la Parte 1, es decir, un coset de un grupo de orden 8192. Cada dominio FRI posterior se obtiene tomando la primera mitad del dominio FRI anterior (abandonando la segunda mitad), y elevando al cuadrado cada uno de sus elementos.<br>

Formalmente - obtuvimos `eval_domain` tomando:<br>
$$w, w\cdot h, w\cdot h^2, ..., w\cdot h^{8191}$$

Por lo tanto, la siguiente capa será:<br>
$$w^2, (w\cdot h)^2, (w\cdot h^2)^2, ..., (w\cdot h^{4095})^2$$

Obsérvese que tomando los cuadrados de la segunda mitad de cada elemento en `eval_domain` produce exactamente
el mismo resultado que tomar los cuadrados de la primera mitad. Lo mismo ocurre con las capas siguientes.
Por ejemplo:

In [ ]:
print(eval_domain[100] ** 2)
half_domain_size = len(eval_domain) // 2
print(eval_domain[half_domain_size + 100] ** 2)

De forma similar, el dominio de la tercera capa será:<br>
$$w^4, (w\cdot h)^4, (w\cdot h^2)^4, ..., (w\cdot h^{2047})^4$$

Y así sucesivamente.

Escribe una función `next_fri_domain` que tome el dominio anterior como argumento, y emita el siguiente.

In [ ]:
def next_fri_domain(fri_domain):
    # Arregla esto.
    return [x for x in fri_domain[:len(fri_domain) // 2]]

Ejecute la prueba: 

In [ ]:
# Prueba usando un hash previamente calculado.
from hashlib import sha256
next_domain = next_fri_domain(eval_domain)
assert '5446c90d6ed23ea961513d4ae38fc6585f6614a3d392cb087e837754bfd32797' == sha256(','.join([str(i) for i in next_domain]).encode()).hexdigest()
print('Éxito!')

### Operador de Plegado FRI (FRI Folding Operator)
El primer polinomio FRI es simplemente el polinomio de composición, es decir, `cp`.<br>
Cada polinomio FRI posterior se obtiene mediante:
1. Obtener un elemento aleatorio del field $\beta$ (llamando a `Channel.receive_random_field_element`).
2. Multiplicando los coeficientes impares del polinomio anterior por $\beta$.
3. Haciendo la suma de pares consecutivos (pares-impares) de coeficientes.

Formalmente, digamos que el polinomio k-ésimo es de grado $< m$ (para algún $m$ que es una potencia de 2):

$$p_{k}(x) := \sum _{i=0} ^{m-1} c_i x^i$$


Entonces el (k+1)-ésimo polinomio, cuyo grado es $< \frac m 2 $ será:

$$p_{k+1}(x) := \sum _{i=0} ^{  m / 2 - 1 } (c_{2i} + \beta \cdot c_{2i + 1}) x^i$$

Escribe una función `next_fri_polynomial` que toma como argumentos un polinomio y un elemento del field (al que nos referíamos como $\beta$), y retorna el siguiente polinomio "plegado".

Tenga en cuenta que:
1. `Polynomial.poly` contiene una lista de los coeficientes de un polinomio, el término independiente primero, y el de mayor grado en último lugar, entonces `p.poly[i] == u` si el coeficiente de $x^i$ es $u$.*
2. El constructor por defecto de `Polynomial`'s  toma la lista de coeficientes como argumento. Así, un polinomio puede instanciarse a partir de una lista de coeficientes `l` llamando a `Polynomial(l)`.


In [ ]:
def next_fri_polynomial(poly, beta):
    odd_coefficients = poly.poly[1::2] # No es necesario corregir esta línea.
    even_coefficients = poly.poly[::2] # Tampoco es necesario arreglar esta línea.
    odd = 'TU_CÓDIGO_AQUÍ'
    even = 'TU_CÓDIGO_AQUÍ'
    return 'TU_CÓDIGO_AQUÍ'

Ejecute la prueba:

In [ ]:
next_p = next_fri_polynomial(cp, FieldElement(987654321))
assert '6bff4c35e1aa9693f9ceb1599b6a484d7636612be65990e726e52a32452c2154' == sha256(','.join([str(i) for i in next_p.poly]).encode()).hexdigest()
print('Éxito!')

### Juntándolo todo para conseguir la siguiente capa FRI

Escribe una función `next_fri_layer` que tome un polinomio, un dominio y un elemento de field (de nuevo - $\beta$), y retorne el siguiente polinomio, el siguiente dominio, y la evaluación de este siguiente polinomio en este siguiente dominio.

In [ ]:
def next_fri_layer(poly, domain, beta):
    next_poly = 'TU_CÓDIGO_AQUÍ'
    next_domain = 'TU_CÓDIGO_AQUÍ'
    next_layer = 'TU_CÓDIGO_AQUÍ'
    return next_poly, next_domain, next_layer

Ejecute la prueba:

In [ ]:
test_poly = Polynomial([FieldElement(2), FieldElement(3), FieldElement(0), FieldElement(1)])
test_domain = [FieldElement(3), FieldElement(5)]
beta = FieldElement(7)
next_p, next_d, next_l = next_fri_layer(test_poly, test_domain, beta)
assert next_p.poly == [FieldElement(23), FieldElement(7)]
assert next_d == [FieldElement(9)]
assert next_l == [FieldElement(86)]
print('Éxito!')

## Generando Compromisos FRI (FRI Commitments)

Hemos desarrollado las herramientas para escribir el método `FriCommit`, que contiene el bucle principal de compromiso FRI.<br>

Toma los siguientes 5 argumentos:
1. El polinomio de composición, que también es el primer polinomio FRI, es decir, `cp`.
2. El coset de orden 8192 que es también el primer dominio FRI, es decir, `eval_domain`.
3. La evaluación de lo primero sobre lo segundo, que también es la primera capa FRI, es decir, `cp_eval`.
4. El primer árbol de Merkle (tendremos uno para cada capa FRI) construido a partir de estas evaluaciones, es decir, `cp_merkle`.
5. Un channel object, es decir, `channel`.

El método retorna 4 listas:
1. Los polinomios FRI.
2. Los dominios FRI.
3. Las capas FRI.
4. Los árboles Merkle FRI.

El método contiene un bucle, en cada iteración ampliamos estas cuatro listas, utilizando el último elemento de cada una de ellas.
La iteración debe detenerse una vez que el último polinomio FRI sea de grado 0, es decir, cuando el último polinomio FRI sea sólo una constante. Entonces debe enviar por el channel esta constante (es decir, el término independiente).
La clase `Channel` sólo admite el envío de cadenas, así que asegúrate de convertir todo lo que desees enviar a través del channel a una cadena antes de enviarlo.

In [ ]:
# Arregla esto según las instrucciones (las líneas sin comentarios específicos están bien).
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while 'TU_CÓDIGO_AQUÍ': # Sustituye esto por la condición de parada correcta.
        beta = 'TU_CÓDIGO_AQUÍ' # Cambie para obtener un elemento aleatorio del channel.
        next_poly, next_domain, next_layer = 'YOUR_CODE_HERE' # Arregle para obtener el siguiente polinomio FRI, dominio y capa.
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append('TU_CÓDIGO_AQUÍ') # Arregle para construir el árbol de Merkle correcto.
        channel.send('TU_CÓDIGO_AQUÍ') # Arregle para enviar el compromiso (commitment) correcto.
    channel.send('TU_CÓDIGO_AQUÍ') # Arregle para enviar el término independiente de la última capa.
    return fri_polys, fri_domains, fri_layers, fri_merkles

Ejecute la prueba:

In [ ]:
test_channel = Channel()
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, test_channel)
assert len(fri_layers) == 11, f'El número esperado de capas FRI es 11, mientras que en realidad es {len(fri_layers)}.'
assert len(fri_layers[-1]) == 8, f'Se espera que la última capa contenga exactamente 8 elementos, contiene {len(fri_layers[-1])}.'
assert all([x == FieldElement(-1138734538) for x in fri_layers[-1]]), f'Se espera que la última capa sea constante.'
assert fri_polys[-1].degree() == 0, 'Se espera que el último polinomio sea constante (grado 0).'
assert fri_merkles[-1].root == '1c033312a4df82248bda518b319479c22ea87bd6e15a150db400eeff653ee2ee', 'La raíz de Merkle de la última capa es incorrecta.'
assert test_channel.state == '61452c72d8f4279b86fa49e9fb0fdef0246b396a4230a2bfb24e2d5d6bf79c2e', 'El estado del channel no es el esperado.'
print('Éxito!')

Corre la siguiente celda para ejecutar la función con tu channel object e imprimir la prueba hasta el momento:

In [ ]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)
print(channel.proof) 